In [1]:
# Create a spark session
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
# Imports neccesary
import pandas as pd
import pydataset

In [ ]:
tips = pydataset.data('tips')
df = spark.createDataFrame(tips)
df

In [ ]:
## 1. Create a spark data frame that contains your favorite programming languages.